<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/20220109_hj_know.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Train Data 불러오기


---

리스트 인덱스별로  
train[0] : 2017 data  
train[1] : 2018 data  
train[2] : 2019 data  
train[3] : 2020 data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import warnings
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('/content/drive/MyDrive/KNOW/train/*.csv'))]

## 전체 데이터 전처리

#### 2017 데이터 전처리

In [3]:
know2017 = know_train[0]

In [4]:
know2017_text = know2017.loc[:,['bq4_1a','bq4_1b','bq4_1c','bq5_2','bq30','bq31','bq32','bq33','bq34','bq38_1']]

In [5]:
def columns_data_processing1(Series):
  return Series.str.replace(r'[^\w]'," ")

def columns_data_processing2(Series):
  return Series.str.replace(' ',",")

def columns_data_processing3(Series):
  return Series.str.replace(',,',",")

def columns_data_processing4(Series):
  return Series.str.replace('없다',"무")

def columns_data_processing5(Series):
  return Series.str.replace('없음',"무")

def columns_data_processing6(Series):
  return Series.str.lower()

def columns_data_processing7(Series):
  return Series.str.replace('등',"")

def columns_data_processing8(Series):
  return Series.str.replace('및',"")

In [6]:
know2017_text = know2017_text.apply(columns_data_processing1,axis=0)
know2017_text = know2017_text.apply(columns_data_processing2,axis=0)
know2017_text = know2017_text.apply(columns_data_processing3,axis=0)
know2017_text = know2017_text.apply(columns_data_processing4,axis=0)
know2017_text = know2017_text.apply(columns_data_processing5,axis=0)
know2017_text = know2017_text.apply(columns_data_processing6,axis=0)

#### 2018 데이터 전처리

In [7]:
know2018 = know_train[1]

In [8]:
know2018_text = know2018.loc[:,['bq4_1a','bq4_1b','bq4_1c','bq5_2','bq29','bq30','bq31','bq32','bq33','bq37','bq37_1']]

In [9]:
know2018_text = know2018_text.apply(columns_data_processing1,axis=0)
know2018_text = know2018_text.apply(columns_data_processing2,axis=0)
know2018_text = know2018_text.apply(columns_data_processing3,axis=0)
know2018_text = know2018_text.apply(columns_data_processing4,axis=0)
know2018_text = know2018_text.apply(columns_data_processing5,axis=0)
know2018_text = know2018_text.apply(columns_data_processing6,axis=0)

#### 2019 데이터 전처리

In [10]:
know2019 = know_train[2]

In [11]:
know2019_text = know2019.loc[:,['bq4_1a','bq4_1b','bq4_1c','bq5_2','bq22','bq23','bq24','bq27','bq27_1']]

In [12]:
know2019_text = know2019_text.apply(columns_data_processing1,axis=0)
know2019_text = know2019_text.apply(columns_data_processing2,axis=0)
know2019_text = know2019_text.apply(columns_data_processing3,axis=0)
know2019_text = know2019_text.apply(columns_data_processing4,axis=0)
know2019_text = know2019_text.apply(columns_data_processing5,axis=0)
know2019_text = know2019_text.apply(columns_data_processing6,axis=0)

#### 2020 데이터 전처리

In [13]:
know2020 = know_train[2]

In [14]:
know2020_text = know2020.loc[:,['bq4_1a','bq4_1b','bq4_1c','bq5_2','bq22','bq23','bq27','bq27_1']]

In [15]:
know2020_text = know2020_text.apply(columns_data_processing1,axis=0)
know2020_text = know2020_text.apply(columns_data_processing2,axis=0)
know2020_text = know2020_text.apply(columns_data_processing3,axis=0)
know2020_text = know2020_text.apply(columns_data_processing4,axis=0)
know2020_text = know2020_text.apply(columns_data_processing5,axis=0)
know2020_text = know2020_text.apply(columns_data_processing6,axis=0)

## 각 Column 별 전처리

### 도구 전처리

In [16]:
tool2017 = know2017_text['bq31']
tool2018 = know2018_text['bq30']

tool = pd.concat([tool2017,tool2018],ignore_index=True)

In [17]:
tool = pd.DataFrame(tool)

In [18]:
tool = tool.values.tolist()

In [19]:
# 도구들을 쉼표 단위로 나눠서 tool list에 담는다.
tools = []
for i in range(0,len(tool)):
  tool_1 = tool[i][0].split(',')
  tools.append(tool_1)

# list를 합친다.
tools = sum(tools,[])

In [20]:
# '' 데이터 삭제.
tools = pd.DataFrame(tools)
index = tools[tools[0] == ''].index
tools = tools.drop(index)

In [21]:
### 라벨 인코딩 ###
tools = tools[0].values.tolist()

# dict에 도구 값 넣기
my_dict = {}
for i in range(len(tools)):
  my_dict[tools[i]] = i


def text_to_list(text:str):
  return list(text.split(','))

know2017_text['bq31'] = know2017_text['bq31'].map(text_to_list)
know2018_text['bq30'] = know2018_text['bq30'].map(text_to_list)

In [22]:
def tool_to_label(tools:list):
  result = []
  for tool in tools:
    if tool in my_dict:
      result.append(my_dict[tool])
    else :
      result.append(0)
  return result

know2017_text['label_tool'] = know2017_text['bq31'].map(tool_to_label)
know2018_text['label_tool'] = know2018_text['bq30'].map(tool_to_label)

### 전공 전처리

##### 2017년도

##### 2018년도

In [23]:
# bq37행에서 전공 걸러내기
know2018_text['bq37'].unique()

# bq37행에서 전공 걸러내서 bq37_1_2018에 담기
bq37_1_2018 = know2018_text[(know2018_text['bq37'] !='1')
&(know2018_text['bq37'] !='2')
&(know2018_text['bq37'] !='3')
&(know2018_text['bq37'] !='4')
&(know2018_text['bq37'] !='5')
&(know2018_text['bq37'] !='6')]

# 잘못된 입력 값 인덱스 찾기
wrong_index = know2018_text[(know2018_text['bq37'] !='1')
&(know2018_text['bq37'] !='2')
&(know2018_text['bq37'] !='3')
&(know2018_text['bq37'] !='4')
&(know2018_text['bq37'] !='5')
&(know2018_text['bq37'] !='6')].index

# 원래 데이터에서 잘못된 row 제거
know2018_text = know2018_text.drop(wrong_index)

In [24]:
# column 이름을 바꿔서 원위치 시키기
bq37_1_2018.rename(columns={'bq37_1':'bq37-1'},inplace=True)
bq37_1_2018.rename(columns={'bq37':'bq37_1'},inplace=True)
bq37_1_2018.rename(columns={'bq37-1':'bq37'},inplace=True)

In [25]:
# 데이터 합치기
know2018_text = pd.concat([know2018_text,bq37_1_2018])

In [26]:
# 데이터 날리기
drop_index = know2018[(know2018_text['bq37'] == '색채심리상담사')].index
know2018_text.drop(drop_index)

,bq4_1a,bq4_1b,bq4_1c,bq5_2,bq29,bq30,bq31,bq32,bq33,bq37,bq37_1,label_tool
0,치과기공사,",",",","세라믹,지르크니아,등,제작,방법,프로그램,사용방법",치기공사,[치카공],",",교정전문가,",",3,치기공과,[25625]
1,정보처리기사,",",",","게임,개발,독학,네트워크,기본,지식",무,"[서브라임텍스트, 유니티, 리눅스]",정보보호컨설턴트,"일반,프로그래머",무,2,이과,"[25626, 44416, 29353]"
2,",",",",",",",",사장님,"[망치, 실리콘, 사다리, 유리칼]",회사원,",",",",2,기계과,"[45374, 45175, 45526, 40006]"
3,화학분석기능사,",",",",",",",","[전자현미경, 컴퓨터]",",",",",",",4,"물리,과학","[43177, 45687]"
4,치과기능사,",",",",",",",","[니켈, 크롬, 코발트]",판매업,",",",",4,치기공학,"[25635, 25636, 25637]"
...,...,...,...,...,...,...,...,...,...,...,...,...
8760,",",",",1,6,통제시스템,"[, ]",",",",",3,1,기계학,"[0, 0]"
8783,산업안전기사,워드프로세서,1,3,엑셀,"[, ]","각종,공기업,공무원",",",4,1,전기공학,"[0, 0]"
8987,",",",",2,5,기기,"[, ]",",",",",2,1,금속,"[0, 0]"
8992,",",",",1,3,철도차량프로그램,"[, ]","철도차량,부품,제작업체",해당무,2,1,기계공학,"[0, 0]"


#### 2019년도

In [27]:
# num to str
know2019_text['bq27'] = know2019_text['bq27'].map(str)

In [28]:
know2019_text['bq27'].unique()

array(['4', '3', '2', '1', '5', '6', '지능로봇학', '모바일인터넷', 'nan', '영상영화디자인과',
       '방사선과', '컴퓨터공학', '신문방송'], dtype=object)

In [29]:
# bq37행에서 전공 걸러내기
know2019_text['bq27'].unique()

# bq37행에서 전공 걸러내서 bq37_1_2018에 담기
bq27_1_2019 = know2019_text[(know2019_text['bq27'] =='컴퓨터공학')
|(know2019_text['bq27'] == '지능로봇학')
|(know2019_text['bq27'] == '모바일인터넷')
|(know2019_text['bq27'] == '영상영화디자인과')
|(know2019_text['bq27'] == '방사선과')
|(know2019_text['bq27'] == '신문방송')
&(know2019_text['bq27'] !='1')
&(know2019_text['bq27'] !='2')
&(know2019_text['bq27'] !='3')
&(know2019_text['bq27'] !='4')
&(know2019_text['bq27'] !='5')
&(know2019_text['bq27'] !='6')]

# 잘못된 입력 값 인덱스 찾기
wrong_index = know2019_text[(know2019_text['bq27'] =='컴퓨터공학')
|(know2019_text['bq27'] == '지능로봇학')
|(know2019_text['bq27'] == '모바일인터넷')
|(know2019_text['bq27'] == '영상영화디자인과')
|(know2019_text['bq27'] == '방사선과')
|(know2019_text['bq27'] == '신문방송')
&(know2019_text['bq27'] !='1')
&(know2019_text['bq27'] !='2')
&(know2019_text['bq27'] !='3')
&(know2019_text['bq27'] !='4')
&(know2019_text['bq27'] !='5')
&(know2019_text['bq27'] !='6')].index

# 원래 데이터에서 잘못된 row 제거
know2019_text = know2019_text.drop(wrong_index)

In [30]:
bq27_1_2019['bq27'].unique()

array(['지능로봇학', '모바일인터넷', '영상영화디자인과', '방사선과', '컴퓨터공학', '신문방송'],
      dtype=object)

In [31]:
# column 이름을 바꿔서 원위치 시키기

bq27_1_2019.rename(columns={'bq27_1':'bq27-1'},inplace=True)
bq27_1_2019.rename(columns={'bq27':'bq27_1'},inplace=True)
bq27_1_2019.rename(columns={'bq27-1':'bq27'},inplace=True)

In [32]:
# 데이터 합치기
know2019_text = pd.concat([know2019_text,bq27_1_2019])

#### 2020년도



---

issue

bq27이 최종학력  
bq27_1이 전공

In [33]:
# bq37행에서 전공 걸러내기
know2020_text['bq27'].unique()

# bq37행에서 전공 걸러내서 bq37_1_2018에 담기
bq27_1_2020 = know2020_text[(know2020_text['bq27'] =='컴퓨터공학')
|(know2020_text['bq27'] == '지능로봇학')
|(know2020_text['bq27'] == '모바일인터넷')
|(know2020_text['bq27'] == '영상영화디자인과')
|(know2020_text['bq27'] == '방사선과')
|(know2020_text['bq27'] == '신문방송')
&(know2020_text['bq27'] !='1')
&(know2020_text['bq27'] !='2')
&(know2020_text['bq27'] !='3')
&(know2020_text['bq27'] !='4')
&(know2020_text['bq27'] !='5')
&(know2020_text['bq27'] !='6')]

# 잘못된 입력 값 인덱스 찾기
wrong_index = know2020_text[(know2020_text['bq27'] =='컴퓨터공학')
|(know2020_text['bq27'] == '지능로봇학')
|(know2020_text['bq27'] == '모바일인터넷')
|(know2020_text['bq27'] == '영상영화디자인과')
|(know2020_text['bq27'] == '방사선과')
|(know2020_text['bq27'] == '신문방송')
&(know2020_text['bq27'] !='1')
&(know2020_text['bq27'] !='2')
&(know2020_text['bq27'] !='3')
&(know2020_text['bq27'] !='4')
&(know2020_text['bq27'] !='5')
&(know2020_text['bq27'] !='6')].index

In [34]:
# 원래 데이터에서 잘못된 row 제거
know2020_text = know2020_text.drop(wrong_index)

In [35]:
# column 이름을 바꿔서 원위치 시키기

bq27_1_2020.rename(columns={'bq27_1':'bq27-1'},inplace=True)
bq27_1_2020.rename(columns={'bq27':'bq27_1'},inplace=True)
bq27_1_2020.rename(columns={'bq27-1':'bq27'},inplace=True)

In [36]:
# 데이터 합치기
know2020_text = pd.concat([know2020_text,bq27_1_2020])

#### 전공 라벨 인코딩

2017 > bq38_1  
2018 > bq37_1  
2019 > bq_27_1  
2020 > bq_27_1 

In [37]:
major2017 = know2017_text['bq38_1']
major2018 = know2018_text['bq37_1']
major2019 = know2019_text['bq27_1']
major2020 = know2020_text['bq27_1']

major = pd.concat([major2017,major2018,major2019,major2020],ignore_index=True)

In [38]:
# 이상값 제거
major.loc[major[0] == '2',0]='이'

major = major.values.tolist()

# 중복 제거
major = set(major)
major = list(major)

major = pd.DataFrame(major)

# null 값 제거
major = major[major[0].notnull()]

In [39]:
major = major.apply(lambda x : x.str.strip('학과'), axis = 1)
major = major.apply(lambda x : x.str.strip('과'), axis = 1)
major = major.apply(lambda x : x.str.strip('학'), axis = 1)
major = major.apply(lambda x : x.str.strip('공'), axis = 1)
major = major.apply(lambda x : x.str.strip('대'), axis = 1)
major = major.apply(lambda x : x.str.strip('부'), axis = 1)

In [40]:
major 

,0
1,농화
2,메카트로닉스
3,상고졸
4,건축설계
5,식음료
...,...
3503,레크레이션
3504,연극영화교육
3505,식품영향
3506,응용화학


In [41]:
major = major.values.tolist()

In [42]:
majors = []
for i in range(0,3507):
  major_1 = major[i][0].split(',') 
  majors.append(major_1)

# list를 합친다.
majors = sum(majors,[])

In [43]:
# '' 데이터 삭제.
majors = pd.DataFrame(majors)
index = majors[majors[0] == ''].index
majors = majors.drop(index)

In [44]:
### 라벨 인코딩 ###
majors = majors[0].values.tolist()
majors = set(majors)
majors = list(majors)

# dict에 도구 값 넣기
my_dict = {}
for i in range(len(majors)):
  my_dict[tools[i]] = i


def text_to_list(text:str):
  return list(text.split(','))

know2017_text['bq38_1'] = know2017_text['bq38_1'].map(text_to_list)

In [47]:
def major_to_label(majors:list):
  result = []
  for major in majors:
    if major in my_dict:
      result.append(my_dict[major])
    else :
      result.append(0)
  return result

know2017_text['major_tool'] = know2017_text['bq38_1'].map(tool_to_label)

In [48]:
know2017_text

,bq4_1a,bq4_1b,bq4_1c,bq5_2,bq30,bq31,bq32,bq33,bq34,bq38_1,label_tool,major_tool
0,자동차도장기능사,",",",",실무교육,무,[무],무,"건설현장,노무직",무,[실업],[0],[0]
1,건축전기설비기술사,",",",",",",무,"[매타기, 드라이버, 가위, cad]",",",",",",",[건축공학],[0],[0]
2,건축전기설비기술사,",",",",",",무,"[건축설계표, 오토캐드, 파이프, 보일러]",",",",",",",[건축공학],[0],[0]
3,지적기사,",",",","측량,및,정보통신기술,교육",지적기사,"[토탈스테이션, szp]",무,지적공무원,",",[환경학과],[0],[0]
4,건축전기설비기술사,",",",",",",무,"[캐드, 엑셀, 건축도면]",",",",",",",[건축공학],[0],[0]
...,...,...,...,...,...,...,...,...,...,...,...,...
9481,",",",",",","편집기술에,원리와,이해능력",무,"[한글, 편집프로그램, 포토샵, 일러스트]",무,마케팅업무,무,[산업디자인],[0],[0]
9482,",",",",",","도면,작성,교육,전자,기초,이해",무,"[auto, cad, , catia, 전문, 특수정비도구, 툴, ]",무,무,무,[우주항공공학],[0],[0]
9483,",",",",",",",","건설,노가대","[볼트, 리벳]",무,무,무,[기계],[2012],[2012]
9484,",",",",",",",",무,[컴퓨터],무,모름,",",[문헌정보학],[0],[0]


In [39]:
mask=df['data'].str.len()==1
df['result']=np.where(mask,df['data']+'과','')
df[df['result'].str.len()==2]

## def data processing 다시